In [1]:
import numpy as np
np.__version__

'1.23.5'

# Часть 4. Операции над массивами и Broadcasting

## Что такое Broadcasting

Ранее мы применяли различные универсальные функции к объектам одной размерности (в случае с бинарными операциями), однако на этом их возможности не заканчиваются. Формально broadcast'ингом называется набор процедур, которые позволяет применять UFuncs для массивов различного размера. Посмотрим на практике, как это работает

In [9]:
# освежим немного память
# массивы одной размерности
x = np.array([1, 2, 3])
y = np.array([5, -3, -7])

print("x + y = ", x + y)

x + y =  [ 6 -1 -4]


In [10]:
# а теперь вернёмся к примеру, который уже был ранее,
# но на него не обратили внимание: добавим к массиву скаляр
# (очевидно, что их размеры не совпадают)
print("x + 7 = ", x + 7)

x + 7 =  [ 8  9 10]


In [11]:
# фактически эта операция эквивалентна следующей
print("x + [7 7 7] = ", x + np.array([7, 7, 7]))

x + [7 7 7] =  [ 8  9 10]


В этом и заключается вся "магия" broadcasting'а: NumPy под капотом без явной дупликации (для оптимизации расхода памяти) "растягивает" скаляр до нужных размеров, чтобы сделать операции совместными. Конечно, broadcasting не всегда можно применить (об этом правилах приведения размеров поговорим позже), а пока давайте посмотрим на ещё пару примеров

In [12]:
m = np.ones((3, 3), dtype=int)
m

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

In [13]:
# приведение одномерного массива к двумерному
m + x

array([[2, 3, 4],
       [2, 3, 4],
       [2, 3, 4]])

In [14]:
z = np.array([2, -4, 1]).reshape(3, 1)
z

array([[ 2],
       [-4],
       [ 1]])

In [15]:
# сложим строчку со столбцом (да, возможно даже такое)
x + z

array([[ 3,  4,  5],
       [-3, -2, -1],
       [ 2,  3,  4]])

## Правила приведения размеров массивов

Как было скзаано ранее, broadcasting - это не панацея. Есть определённые правила и ограничения, которые нужно деражть в голове, чтобу получить тот результат, который соответствует цели и ожиданиям.

1) Если два массива отличаются по количеству измерений, то форма массива с меньшим количеством измерений дополняется единицами с ведущей (левой) стороны
2) Если форма двух массивов не совпадает в каком-либо измерении, то массив с формой, равной 1 в этом измерении, растягивается до соответствия другой форме
3) Если в каком-либо измерении размеры не совпадают и ни один из них не равен 1, то выдается ошибка

Применяя описанные выше правила, размеры массивов либо будут приведены к форме, допускающей проведение опреации, либо выдастся сообщение о соответствующей ошибке

## Примеры работы

### Пример 1

In [16]:
x = np.ones((2, 3), dtype=int)
y = np.arange(3, dtype=int)

print("x.shape: ", x.shape)
print("y.shape: ", y.shape)

x.shape:  (2, 3)
y.shape:  (3,)


Что будет происходить? Рассмотрим по порядку:

1) размеры массивов не совпадают => согласно правилу (1) размер массив с меньшим количеством измерений дополняется единицами слева, т.е. `y.shape => (1, 3)`
2) согласно правилу (2) мы растягиваем размер первого массива вдоль несогласованного измерения, т.е. `y.shape => (2, 3)`

In [17]:
 # финально
x + y

array([[1, 2, 3],
       [1, 2, 3]])

### Пример 2

In [18]:
x = np.arange(3, dtype=int).reshape((3, 1))
y = np.arange(3, dtype=int)

print("x.shape: ", x.shape)
print("y.shape: ", y.shape)

x.shape:  (3, 1)
y.shape:  (3,)


Вновь обратимся к правилам:

1) согласно правилу (1) размер массив с меньшим количеством измерений дополняется единицами слева, т.е. `y.shape => (1, 3)`
2) согласно правилу (2) ОБА массива растягиваются вдоль тех измерений, где есть различия, т.е. `x.shape => (3, 3)` и `y.shape => (3, 3)`

In [19]:
 # финально
x + y

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

### Пример 3

In [20]:
x = np.ones((3, 2), dtype=int)
y = np.arange(3)

print("x.shape: ", x.shape)
print("y.shape: ", y.shape)

x.shape:  (3, 2)
y.shape:  (3,)


Смотрим в очередной раз:

1) согласно правилу (1) размер массив с меньшим количеством измерений дополняется единицами слева, т.е. `y.shape => (1, 3)`
2) согласно правилу (2) массив `y` растягивается вдоль тех измерений, где есть различия, т.е. `y.shape => (3, 3)`

На выходе получаем `x.shape = (3, 2)` и `y.shape => (3, 3)`, что даёт нам невозможность провести broadcasting по правилу (3)

In [21]:
try:
    x + y
except Exception as e:
    print("ERROR:\n", e)

ERROR:
 operands could not be broadcast together with shapes (3,2) (3,) 
